# Carla Intro

Carla Python API is a Python library that allows you to control a Carla simulation environment.

You can find the official documentation [here](https://carla.readthedocs.io/en/latest/python_api/).

In [14]:
import carla
import time

## Carla client connetion

A connetion to the server will be created. From now, we can use the client to interact with the server.

> **Remember**: *Client* object is a proxy to the server. It is used to send commands to the server and receive responses.

In [3]:
client = carla.Client("isi-simcar.campusfc.dir.unibo.it", 2000)

As a remote proxy, connection to the server is not immediate. It may take a few seconds to establish the connection.

In [ ]:
client.set_timeout(0.5)
client.load_world("Town01") # a time consuming operation

In [ ]:
client.set_timeout(15)
client.load_world("Town02")

In [ ]:
print(f"Client version: {client.get_client_version()}")
print(f"Server version: {client.get_server_version()}")

## Configure the simulation

Client is also used to configure the simulation. We can set the weather, the time of the day, the map, etc.

### Change maps

More maps can be downloaded from [GitHub Releases](https://github.com/carla-simulator/carla/releases).

In [ ]:
client.get_available_maps()

`Town10HD_Opt` is a streamlined version of `Town10HD` designed to balance performance with environmental detail.

In [ ]:
client.load_world("Town10HD_Opt")

### Set the weather

In [36]:
world = client.get_world()
world.set_weather(carla.WeatherParameters.WetNight)

In [ ]:
weathers = [
    carla.WeatherParameters.CloudyNoon,
    carla.WeatherParameters.MidRainSunset,
    carla.WeatherParameters.HardRainNoon,
    carla.WeatherParameters.ClearSunset,
    carla.WeatherParameters.SoftRainNight,
    carla.WeatherParameters.Default,
]

for weather in weathers:
    world.set_weather(weather)
    print(f"Weather set to {weather}")
    time.sleep(5)

### Sync vs Async mode

In **synchronous** mode, the server waits for the client to send a tick command before moving to the next frame. This is useful when you want to control the simulation frame by frame.

In **asynchronous** mode, the server runs at its own pace. This is useful when you want to run the simulation as fast as possible.

![Sync vs Async](./img/sync-async.png)

In [ ]:
world = client.get_world()
settings = world.get_settings()
settings.synchronous_mode = True
world.apply_settings(settings)

In [ ]:
count = 0
while True:
    world.tick()
    time.sleep(0.1)
    count += 1
    print(f"Ticked the server {count}", end="\r")

In [ ]:
settings.synchronous_mode = False
world.apply_settings(settings)
client.reload_world()

## Exercise: some config

In this exercise, we will connect to the server, configure the simulation, and run it for a few seconds.

The main goal is to get familiar with the Carla Python API.

### Proposed exercise

Try to implement sun position control similar to the reality.

![Sun position](./img/sun-position.png)

## Add actors to the simulation

### Coordinate system

Coordinates uses 3 axes: `x`, `y`, `z`, in meters.

There is also a rotation system with 3 angles: `pitch`, `yaw`, `roll`, in degrees.

In [27]:
spectator = world.get_spectator()
spectator.set_transform(
    carla.Transform(
        carla.Location(x=0, y=10, z=0),
        carla.Rotation(yaw=0, pitch=0, roll=0)
    )
)

### Spawnpoints

Spawnpoints are used to place actors in the simulation, like vehicles, pedestrians, sensors, etc.

In [ ]:
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
spectator = world.get_spectator()

for spawn_point in spawn_points:
    spectator.set_transform(spawn_point)

    print(spawn_point.location, end="\r")
    time.sleep(5)

### Actors

In [ ]:
blue_prints = world.get_blueprint_library().filter('*')

for blue_print in blue_prints:
    print(f"Id: {blue_print.id}, Tags: {blue_print.tags}")

### Spawn a vehicles

In [35]:
vehicle = world.try_spawn_actor(blue_prints[0], spawn_points[0])

In [37]:
vehicle.set_autopilot(True)

In [ ]:
vehicle.destroy()